# Reasoning without Observation (using the open source Mixtral8x7b model running on Fireworks AI)

See [rewoo.ipynb](./rewoo.ipynb) for a complete discussion of this technique. This notebook only modifies that example with specific changes required to use an open source model, instead of OpenAI.

In [8]:
%pip install -U --quiet langgraph langchain_community fireworks_ai tavily-python

Note: you may need to restart the kernel to use updated packages.


In [9]:
import os
import getpass


def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}=")


os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "ReWOO"
_set_if_undefined("TAVILY_API_KEY")
_set_if_undefined("LANGCHAIN_API_KEY")
_set_if_undefined("OPENAI_API_KEY")

**Graph State**: In LangGraph, every node updates a shared graph state. The state is the input to any node whenever it is invoked.

Below, we will define a state dict to contain the task, plan, steps, and other variables.

In [10]:
from typing import TypedDict, List


class ReWOO(TypedDict):
    task: str
    plan_string: str
    steps: List
    results: dict
    result: str

## 1. Planner

In [15]:
from langchain_community.chat_models import ChatFireworks

model = ChatFireworks(
    model="accounts/fireworks/models/mixtral-8x7b-instruct",  # input context limit is 32k tokens
    model_kwargs={
        "context_length_exceeded_behavior": "truncate",
        "temperature": 0,
        "max_tokens": 32 * 1024,  # this sets the total token max (input and output)
    },
)

In [12]:
prompt = """For the following task, make plans that can solve the problem step by step. For each plan, indicate \
which external tool together with tool input to retrieve evidence. You can store the evidence into a \
variable #E that can be called by later tools. (Plan, #E1, Plan, #E2, Plan, ...)

Tools can be one of the following:
(1) Google[input]: Worker that searches results from Google. Useful when you need to find short
and succinct answers about a specific topic. The input should be a search query.
(2) LLM[input]: A pretrained LLM like yourself. Useful when you need to act with general
world knowledge and common sense. Prioritize it when you are confident in solving the problem
yourself. Input can be any instruction.

For example,
Task: Thomas, Toby, and Rebecca worked a total of 157 hours in one week. Thomas worked x
hours. Toby worked 10 hours less than twice what Thomas worked, and Rebecca worked 8 hours
less than Toby. How many hours did Rebecca work?
Plan: Given Thomas worked x hours, translate the problem into algebraic expressions and solve
with Wolfram Alpha. #E1 = WolframAlpha[Solve x + (2x − 10) + ((2x − 10) − 8) = 157]
Plan: Find out the number of hours Thomas worked. #E2 = LLM[What is x, given #E1]
Plan: Calculate the number of hours Rebecca worked. #E3 = Calculator[(2 ∗ #E2 − 10) − 8]

Begin! 
Describe your plans with rich details. Each Plan should be followed by only one #E.

Task: {task}"""

In [16]:
task = "what is the hometown of the 2024 australian open winner"

In [17]:
result = model.invoke(prompt.format(task=task))

In [18]:
print(result.content)

Plan: Since the 2024 Australian Open winner hasn't been determined yet, I will search for the location of the 2024 Australian Open. #E1 = Google[2024 Australian Open location]
Plan: Utilize LLM to determine the hometown of the 2024 Australian Open winner based on the location of the tournament. #E2 = LLM[What is the hometown of a typical winner of the 2024 Australian Open, given #E1?]

(Note: Since the specific winner isn't known, I'll provide a general answer based on common sense and general knowledge.)


#### Planner Node

In [19]:
import re
from langchain_core.prompts import ChatPromptTemplate

# Regex to match expressions of the form E#... = ...[...]
regex_pattern = r"Plan:\s*(.+)\s*(#E\d+)\s*=\s*(\w+)\s*\[([^\]]+)\]"
prompt_template = ChatPromptTemplate.from_messages([("user", prompt)])
planner = prompt_template | model


def get_plan(state: ReWOO):
    task = state["task"]
    result = planner.invoke({"task": task})
    # Find all matches in the sample text
    matches = re.findall(regex_pattern, result.content)
    return {"steps": matches, "plan_string": result.content}

## 2. Executor

In [20]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

In [21]:
def _get_current_task(state: ReWOO):
    if state["results"] is None:
        return 1
    if len(state["results"]) == len(state["steps"]):
        return None
    else:
        return len(state["results"]) + 1


def tool_execution(state: ReWOO):
    """Worker node that executes the tools of a given plan."""
    _step = _get_current_task(state)
    _, step_name, tool, tool_input = state["steps"][_step - 1]
    _results = state["results"] or {}
    for k, v in _results.items():
        tool_input = tool_input.replace(k, v)
    if tool == "Google":
        result = search.invoke(tool_input)
    elif tool == "LLM":
        result = model.invoke(tool_input)
    else:
        raise ValueError
    _results[step_name] = str(result)
    return {"results": _results}

## 3. Solver

In [22]:
solve_prompt = """Solve the following task or problem. To solve the problem, we have made step-by-step Plan and \
retrieved corresponding Evidence to each Plan. Use them with caution since long evidence might \
contain irrelevant information.

{plan}

Now solve the question or task according to provided Evidence above. Respond with the answer
directly with no extra words.

Task: {task}
Response:"""


def solve(state: ReWOO):
    plan = ""
    for _plan, step_name, tool, tool_input in state["steps"]:
        _results = state["results"] or {}
        for k, v in _results.items():
            tool_input = tool_input.replace(k, v)
        plan += f"Plan: {_plan}\n{step_name} = {tool}[{tool_input}]"
    prompt = solve_prompt.format(plan=plan, task=state["task"])
    result = model.invoke(prompt)
    return {"result": result.content}

## 4. Define Graph

In [23]:
def _route(state):
    _step = _get_current_task(state)
    if _step is None:
        # We have executed all tasks
        return "solve"
    else:
        # We are still executing tasks, loop back to the "tool" node
        return "tool"

In [24]:
from langgraph.graph import StateGraph, END

graph = StateGraph(ReWOO)
graph.add_node("plan", get_plan)
graph.add_node("tool", tool_execution)
graph.add_node("solve", solve)
graph.add_edge("plan", "tool")
graph.add_edge("solve", END)
graph.add_conditional_edges("tool", _route)
graph.set_entry_point("plan")

app = graph.compile()

In [27]:
for s in app.stream({"task": task}):
    print(s)
    print("---")

{'plan': {'steps': [("Since the 2024 Australian Open winner hasn't been determined yet, we cannot know the exact hometown of the winner. However, we can research the past Australian Open winners and their hometowns to provide some context. ", '#E1', 'Google', 'Past Australian Open winners'), ('Use LLM to analyze the search results and identify the hometowns of a few past Australian Open winners. ', '#E2', 'LLM', 'What are the hometowns of some past Australian Open winners, given #E1?'), ('Summarize the hometowns of past Australian Open winners found in #E2. ', '#E3', 'LLM', 'Can you summarize the hometowns of past Australian Open winners you found in #E2?'), ('Present the summary of hometowns of past Australian Open winners as a general context for the 2024 winner. ', '#E4', 'LLM', 'Here are the hometowns of some past Australian Open winners: #E3. The hometown of the 2024 winner is yet to be determined.')], 'plan_string': "Plan: Since the 2024 Australian Open winner hasn't been determi

In [26]:
# Print out the final result
print(s[END]["result"])

Melbourne


## Conclusion

Congratulations on implementing ReWOO! Before you leave, I'll leave you with a couple limitations of the current implementation from the paper:

1. If little context of the environment is available, the planner will be ineffective in its tool use. This can typically be ameliorated through few-shot prompting and/or fine-tuning.
2. The tasks are still executed in sequence, meaning the total execution time is impacted by _every_ tool call, not just he longest-running in a given step.